In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

%matplotlib inline

import argparse
from tqdm import tqdm

import torch
import torch.nn.functional as F
from torch_sparse import SparseTensor
from torch_geometric.utils import to_undirected, dropout_adj
import os
import os.path as osp

from ogb.nodeproppred import PygNodePropPredDataset

import torch
from torch_geometric.data import Data

import os.path as osp
from sklearn import preprocessing

## Загрузка данных

In [2]:
data = pd.read_csv('./Data/2_full_table_data_GNN.csv')

In [3]:
data.shape

(115525128, 14)

In [4]:
data

,date_key,amount,client_inn,partner_inn,years,Длина_Отн_в_днях,Клиент_Сегмент_КБ,Клиент_Сегмент_МБ,Клиент_Сегмент_МКП,Клиент_Сегмент_СБ,Клиент_НаличиеЗПпроекта_Д,Клиент_НаличиеЗПпроекта_Н,Fraud_presence,Reject_presence
0,2018-03-01,41000.00,1279230,1,54.0,6251.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2018-03-01,482300.00,1224261,13022,40.0,1618.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,2018-03-01,15.72,1549773,119556,52.0,7370.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,2018-03-01,4217742.30,976651,976651,51.0,2103.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4,2018-03-01,11.77,1413113,1003953,41.0,526.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115525123,2021-03-01,885.61,1739204,656965,37.0,3182.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
115525124,2021-03-01,127165.00,426567,1796124,35.0,3592.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
115525125,2021-03-01,136920.00,1070950,1736097,51.0,3466.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
115525126,2021-03-01,6.00,1387950,119556,47.0,2519.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


## Нужно ли убирать в данных для x client_inn и partner_inn???

In [43]:
data_with_inn = data

In [44]:
#data = data.drop(['client_inn', 'partner_inn'], axis=1)

In [45]:
data

,date_key,amount,client_inn,partner_inn,years,Длина_Отн_в_днях,Клиент_Сегмент_КБ,Клиент_Сегмент_МБ,Клиент_Сегмент_МКП,Клиент_Сегмент_СБ,Клиент_НаличиеЗПпроекта_Д,Клиент_НаличиеЗПпроекта_Н,Fraud_presence,Reject_presence
0,2018-03-01,41000.00,1279230,1,54.0,6251.0,1,0,0,0,0,1,0.0,0.0
1,2018-03-01,618280.00,1279230,1003953,54.0,6251.0,1,0,0,0,0,1,0.0,0.0
2,2018-03-01,2698540.00,1279230,682959,54.0,6251.0,1,0,0,0,0,1,0.0,0.0
3,2018-03-01,3000.00,1279230,1852820,54.0,6251.0,1,0,0,0,0,1,0.0,0.0
4,2018-03-01,1000.00,1279230,119556,54.0,6251.0,1,0,0,0,0,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7015932,2021-03-01,1000.00,673145,1003953,47.0,127.0,0,0,1,0,0,1,0.0,0.0
7015933,2021-03-01,290.00,764479,119556,46.0,375.0,0,0,1,0,0,1,0.0,1.0
7015934,2021-03-01,2000.00,398315,119556,34.0,126.0,0,1,0,0,0,1,0.0,0.0
7015935,2021-03-01,10855.68,1095817,1095817,42.0,187.0,0,0,1,0,1,0,0.0,0.0


In [46]:
data.shape

(7015937, 14)

In [47]:
#data_with_inn.drop_duplicates(subset=['client_inn'])

## Перевод исходного датасета в датасет для графов

In [5]:
features_dataset = data.drop(['Fraud_presence', 'date_key'], axis=1)

In [6]:
features_dataset

,amount,client_inn,partner_inn,years,Длина_Отн_в_днях,Клиент_Сегмент_КБ,Клиент_Сегмент_МБ,Клиент_Сегмент_МКП,Клиент_Сегмент_СБ,Клиент_НаличиеЗПпроекта_Д,Клиент_НаличиеЗПпроекта_Н,Reject_presence
0,41000.00,1279230,1,54.0,6251.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,482300.00,1224261,13022,40.0,1618.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,15.72,1549773,119556,52.0,7370.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,4217742.30,976651,976651,51.0,2103.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,11.77,1413113,1003953,41.0,526.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
115525123,885.61,1739204,656965,37.0,3182.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
115525124,127165.00,426567,1796124,35.0,3592.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
115525125,136920.00,1070950,1736097,51.0,3466.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
115525126,6.00,1387950,119556,47.0,2519.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [7]:
features_dataset.shape

(115525128, 12)

## Label encoding для client_inn и partner_inn

In [8]:
#Выделяем два столбца: client_inn и partner_inn
client_inn_columnn = features_dataset['client_inn']
partner_inn_columnn = features_dataset['partner_inn']

In [9]:
#Объединяем их один под другим, чтобы получился один столбец
inn_cl_par = pd.concat([client_inn_columnn, partner_inn_columnn], axis=0)
inn_cl_par

0            1279230
1            1224261
2            1549773
3             976651
4            1413113
              ...   
115525123     656965
115525124    1796124
115525125    1736097
115525126     119556
115525127     503750
Length: 231050256, dtype: int64

In [10]:
#Удаляем дубликаты
inn_cl_par = inn_cl_par.drop_duplicates()
inn_cl_par

0            1279230
1            1224261
2            1549773
3             976651
4            1413113
              ...   
115524744    1245863
115524794    1150926
115524795    1599973
115525041    1742304
115525112     782074
Length: 1537517, dtype: int64

In [11]:
#Создаем label encoder на объединенных client_inn и partner_inn
encoder = preprocessing.LabelEncoder()
encoder.fit(inn_cl_par)

LabelEncoder()

In [12]:
#Кодируем и столбец client_inn и столбец partner_inn при помощи Label encoder
features_dataset['new_client_inn'] = encoder.transform(features_dataset['client_inn'])

In [13]:
features_dataset['new_partner_inn'] = encoder.transform(features_dataset['partner_inn'])

In [14]:
features_dataset

,amount,client_inn,partner_inn,years,Длина_Отн_в_днях,Клиент_Сегмент_КБ,Клиент_Сегмент_МБ,Клиент_Сегмент_МКП,Клиент_Сегмент_СБ,Клиент_НаличиеЗПпроекта_Д,Клиент_НаличиеЗПпроекта_Н,Reject_presence,new_client_inn,new_partner_inn
0,41000.00,1279230,1,54.0,6251.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1025066,0
1,482300.00,1224261,13022,40.0,1618.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,982241,10323
2,15.72,1549773,119556,52.0,7370.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1242692,95545
3,4217742.30,976651,976651,51.0,2103.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,786977,786977
4,11.77,1413113,1003953,41.0,526.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1129533,808404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115525123,885.61,1739204,656965,37.0,3182.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1391164,530732
115525124,127165.00,426567,1796124,35.0,3592.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,339497,1437180
115525125,136920.00,1070950,1736097,51.0,3466.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,863351,1388607
115525126,6.00,1387950,119556,47.0,2519.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1108693,95545


In [15]:
features_dataset = features_dataset.drop(['client_inn', 'partner_inn'], axis=1)

In [16]:
features_dataset

,amount,years,Длина_Отн_в_днях,Клиент_Сегмент_КБ,Клиент_Сегмент_МБ,Клиент_Сегмент_МКП,Клиент_Сегмент_СБ,Клиент_НаличиеЗПпроекта_Д,Клиент_НаличиеЗПпроекта_Н,Reject_presence,new_client_inn,new_partner_inn
0,41000.00,54.0,6251.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1025066,0
1,482300.00,40.0,1618.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,982241,10323
2,15.72,52.0,7370.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1242692,95545
3,4217742.30,51.0,2103.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,786977,786977
4,11.77,41.0,526.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1129533,808404
...,...,...,...,...,...,...,...,...,...,...,...,...
115525123,885.61,37.0,3182.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1391164,530732
115525124,127165.00,35.0,3592.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,339497,1437180
115525125,136920.00,51.0,3466.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,863351,1388607
115525126,6.00,47.0,2519.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1108693,95545


In [17]:
x = torch.tensor(features_dataset.drop(['new_client_inn', 'new_partner_inn'], axis=1).values.astype(np.float32)) 

In [18]:
y = torch.tensor(data['Fraud_presence'].values.astype(np.float32))

In [19]:
edge_index = features_dataset[['new_client_inn', 'new_partner_inn']].values.T
edge_index = torch.tensor(edge_index).to(dtype=torch.long)

In [20]:
edge_index

tensor([[1025066,  982241, 1242692,  ...,  863351, 1108693,  609031],
        [      0,   10323,   95545,  ..., 1388607,   95545,  403960]])

In [21]:
#Перевод данных в датасет для графа
data_graph = Data(x=x, y=y, edge_index=edge_index)

In [22]:
data_graph

Data(x=[115525128, 10], edge_index=[2, 115525128], y=[115525128])

In [23]:
x = data_graph.x
N = data_graph.num_nodes

In [24]:
print(N)

115525128


In [25]:
data_graph.edge_index

tensor([[1025066,  982241, 1242692,  ...,  863351, 1108693,  609031],
        [      0,   10323,   95545,  ..., 1388607,   95545,  403960]])

In [26]:
data_graph.edge_index.shape

torch.Size([2, 115525128])

In [27]:
path = './adj_gcn.pt'
print('Making the graph undirected.')

data_graph.edge_index, _ = dropout_adj(
       data_graph.edge_index, p=0, num_nodes=data_graph.num_nodes)

data_graph.edge_index

Making the graph undirected.


/data/home/krazheva/.conda/envs/py310/lib/python3.10/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)


tensor([[1025066,  982241, 1242692,  ...,  863351, 1108693,  609031],
        [      0,   10323,   95545,  ..., 1388607,   95545,  403960]])

In [28]:
data_graph

Data(x=[115525128, 10], edge_index=[2, 115525128], y=[115525128])

In [29]:
data_graph.num_nodes

115525128

In [30]:
data_graph.edge_index.shape

torch.Size([2, 115525128])

In [31]:
data_graph.edge_index = to_undirected(data_graph.edge_index, num_nodes=data_graph.num_nodes)

In [32]:
print(data_graph.edge_index)

tensor([[      0,       0,       0,  ..., 1537514, 1537515, 1537516],
        [      0,      79,      91,  ..., 1158724,  998194, 1195941]])


In [33]:
type(data_graph.edge_index)

torch.Tensor

In [34]:
torch.max(data_graph.edge_index)

tensor(1537516)

In [35]:
row, col = data_graph.edge_index
print('Computing adj...')

adj = SparseTensor(row=row, col=col, sparse_sizes=(N, N))
adj = adj.set_diag()
deg = adj.sum(dim=1).to(torch.float)
deg_inv_sqrt = deg.pow(-0.5)
deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
adj = deg_inv_sqrt.view(-1, 1) * adj * deg_inv_sqrt.view(1, -1)
adj = adj.to_scipy(layout='csr')

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)
adj=sparse_mx_to_torch_sparse_tensor(adj)
print('Start processing')

Computing adj...
Start processing


In [38]:
num_hops = 3
#saved = torch.cat((x[train_idx], x[valid_idx], x[test_idx]), dim=0)
torch.save(data_graph, f'./2_graph.pt')

for i in tqdm(range(num_hops)):
    x = adj @ x
    torch.save(x, f'./2_graph_hope_{i+1}.pt')

100%|██████████| 3/3 [01:27<00:00, 29.07s/it]


In [40]:
import matplotlib.pyplot as plt
import torch
from sklearn.manifold import TSNE

from torch_geometric.datasets import Planetoid
from torch_geometric.nn import Node2Vec
#from torch_geometric.typing import WITH_TORCH_CLUSTER

#if not WITH_TORCH_CLUSTER:
#    quit("This example requires 'torch-cluster'")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cpu"

data = data_graph
model = Node2Vec(data.edge_index, embedding_dim=128, walk_length=20,
                     context_size=10, walks_per_node=10,
                     num_negative_samples=1, p=1, q=1, sparse=True).to(device)

num_workers = 0 if sys.platform.startswith('win') else 4
loader = model.loader(batch_size=128, shuffle=True,
                       num_workers=num_workers)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

@torch.no_grad()
def test():
    model.eval()
    z = model()
    acc = model.test(z[data.train_mask], data.y[data.train_mask],
                     z[data.test_mask], data.y[data.test_mask],
                     max_iter=150)
    return acc

for epoch in range(1, 101):
    loss = train()
    acc = test()
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Acc: {acc:.4f}')

@torch.no_grad()
def plot_points(colors):
    model.eval()
    z = model(torch.arange(data.num_nodes, device=device))
    z = TSNE(n_components=2).fit_transform(z.cpu().numpy())
    y = data.y.cpu().numpy()

    plt.figure(figsize=(8, 8))
    for i in range(dataset.num_classes):
        plt.scatter(z[y == i, 0], z[y == i, 1], s=20, color=colors[i])
    plt.axis('off')
    plt.show()

colors = [
    '#ffc0cb', '#bada55', '#008080', '#420420', '#7fe5f0', '#065535',
    '#ffd700'
]
plot_points(colors)


if __name__ == "__main__":
    main()

NameError: name 'sys' is not defined